In [1]:
# Execute if necessary
# %%capture
# !pip install numpy seaborn matplotlib pandas

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, Tuple, Union, List

# Práctica 3: Modelo Lineal

__Instrucciones__: A continuación hay una lista de funciones que debe implementar o tareas que debe desarrollar. La descripción de cada una de ellas se encuentra en la definición de cada una de las funciones. Cada función está marcada por &#x1F625;,  &#x1F643; o &#x1F921;. Las marcas indican:

- &#x1F625;: Indican una entrega que debe ser hecha dentro de la misma sesión de la asignación. 
- &#x1F643;: Indican una entrega que puede ser hecha hasta la siguiente sesión.
- &#x1F921;: Debe mostrar un avance en la misma sesión, pero la entrega puede ser hecha en la siguiente.

Aquellas entregas parciales que no sean hechas el día de la asignación ya no serán válidas para las entregas totales, sin embargo, las entregas totales seguirán siendo válidas.

En esta sección se incluye un dataset real. El dataset importado incluye multiples características que describen las condiciones de los pasajeros en el accidente del titanic.

- __PassengerId__: Identificador de cada pasajero.
- __Survived__: 0 si no sobrevivió al accidente, 1 si lo hizo.
- __Pclass__: Clase en la que viajaba el pasajero, 1 - Primera clase, 2 - Segunda clase y 3 - Tercera clase.
- __Name__: Nombre del pasajero.
- __Sex__: Sexo del pasajero.
- __Age__: Edad del pasajero.
- __SibSp__: Número de hermanos más número de esposas con las que viajaba el pasajero.
- __Parch__: Número de padres más número de hijos con las que viajaba el pasajero.
- __Ticket__: Número de boleto.
- __Fare__: Tarifa del boleto del pasajero.
- __Cabin__: Número de cabina del pasajero.
- __Embarked__: Puerto de embarcación, C - Cherbourg, Q - Queenstown y S - Southampton.


In [3]:
df = pd.read_csv("titanic.csv")
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


## Asignación 1 &#x1F625;

Realice el preprocesamiento que considere adecuado para que las características __Pclass__, __Sex__, __SibSp__, __Parch__, __Fare__, __Cabin__, __Embarked__ y __Survived__ puedan ser utilizadas por un modelo lineal.

In [4]:
selected_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Survived']
df_selected = df[selected_features]

# Imputar los valores faltantes en la columna 'Fare'
fare_median = df_selected['Fare'].median()
df_selected['Fare'] = df_selected['Fare'].fillna(fare_median)

# Crear una nueva columna para indicar si el pasajero tenía o no asignada una cabina
df_selected['CabinAssigned'] = df_selected['Cabin'].notnull().astype(int)

# Eliminar la columna original de Cabin
df_selected.drop(['Cabin'], axis=1, inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13436\2975337610.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Fare'] = df_selected['Fare'].fillna(fare_median)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13436\2975337610.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['CabinAssigned'] = df_selected['Cabin'].notnull().astype(int)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13436\2975337610.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [5]:
# Convertir las características categóricas en variables dummy
sex_dummies = pd.get_dummies(df_selected['Sex'], prefix='Sex')
embarked_dummies = pd.get_dummies(df_selected['Embarked'], prefix='Embarked')
df_selected = pd.concat([df_selected, sex_dummies, embarked_dummies], axis=1)
df_selected.drop(['Sex', 'Embarked'], axis=1, inplace=True)

In [6]:
# Normalizar las características numéricas
numeric_features = ['Pclass', 'SibSp', 'Parch', 'Fare']
for feature in numeric_features:
    feature_mean = df_selected[feature].mean()
    feature_std = df_selected[feature].std()
    df_selected[feature] = (df_selected[feature] - feature_mean) / feature_std

print(df_selected)

       Pclass     SibSp     Parch      Fare  Survived  CabinAssigned  \
0    0.826913  0.432550 -0.473408 -0.502163         0              0   
1   -1.565228  0.432550 -0.473408  0.786404         1              1   
2    0.826913 -0.474279 -0.473408 -0.488580         1              0   
3   -1.565228  0.432550 -0.473408  0.420494         1              1   
4    0.826913 -0.474279 -0.473408 -0.486064         0              0   
..        ...       ...       ...       ...       ...            ...   
886 -0.369158 -0.474279 -0.473408 -0.386454         0              0   
887 -1.565228 -0.474279 -0.473408 -0.044356         1              1   
888  0.826913  0.432550  2.007806 -0.176164         0              0   
889 -1.565228 -0.474279 -0.473408 -0.044356         1              1   
890  0.826913 -0.474279 -0.473408 -0.492101         0              0   

     Sex_female  Sex_male  Embarked_C  Embarked_Q  Embarked_S  
0             0         1           0           0           1  
1      

## Asignación 2 &#x1F625;

Utilizando las características __Pclass__, __Sex__, __SibSp__, __Parch__, __Fare__, __Cabin__ y __Embarked__, entrene un clasificador lineal para predecir __Survived__ utilizando el algoritmo _pocket_. Imprima el error obtenido.

In [7]:
# Paso 1: Importar bibliotecas
from sklearn.model_selection import train_test_split

# Paso 2: Cargar datos
data = pd.read_csv("titanic.csv")

# Paso 3: Preprocesar datos
data = data.drop(["PassengerId", "Name", "Ticket"], axis=1)
data["Sex"] = pd.factorize(data["Sex"])[0]
data["Cabin"] = pd.factorize(data["Cabin"])[0]
data["Embarked"] = pd.factorize(data["Embarked"])[0]
data = data.fillna(data.mean())

# Paso 4: Separar datos en conjuntos de entrenamiento y prueba
X = data.drop("Survived", axis=1)
y = data["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Paso 5: Definir función de entrenamiento del clasificador
def train(X_train, y_train, X_test, y_test, max_iter=1000):
    # Agregar una columna de unos para el término de sesgo
    X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
    X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

    # Inicializar pesos aleatorios
    w = np.random.randn(X_train.shape[1])

    # Entrenar el clasificador con el algoritmo pocket
    best_w = w
    best_error = np.inf
    error = np.zeros(max_iter)
    for i in range(max_iter):
        y_pred = np.sign(np.dot(X_train, w))
        y_pred[y_pred == 0] = -1
        error[i] = np.mean(y_pred != y_train)

        if error[i] < best_error:
            best_w = w.copy()
            best_error = error[i]

        # Actualizar pesos con el algoritmo de perceptrón
        idx = np.where(y_pred != y_train)[0]
        if len(idx) == 0:
            break
        x = X_train[idx[0]]
        y_true = y_train[idx[0]]
        w = w + y_true * x

    # Calcular error en conjunto de prueba
    y_pred_test = np.sign(np.dot(X_test, best_w))
    y_pred_test[y_pred_test == 0] = -1
    test_error = np.mean(y_pred_test != y_test)

    return best_w, best_error, test_error, error

# Paso 6: Entrenar el clasificador y calcular error de entrenamiento y prueba
w, train_error, test_error, error = train(X_train, y_train, X_test, y_test)

# Imprimir resultados
print("Error de entrenamiento:", train_error)
print("Error de prueba:", test_error)

Error de entrenamiento: 0.6938202247191011
Error de prueba: 0.7150837988826816


## Asignación 3 &#x1F625;

Utilizando las características __Pclass__, __Sex__, __SibSp__, __Parch__, __Cabin__ y __Embarked__, entrene una regresión lineal para predecir __Fare__ utilizando el algoritmo de Ordinary Leasts Squares (OLS). Imprima el valor del error cuadrático medio (MSE).

In [8]:
# Selección de características y preprocesamiento de datos
selected_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Cabin', 'Embarked', 'Fare']
df_selected = df[selected_features]

df_selected['CabinAssigned'] = df_selected['Cabin'].notnull().astype(int)

df_selected.drop(['Cabin'], axis=1, inplace=True)

sex_dummies = pd.get_dummies(df_selected['Sex'], prefix='Sex')
embarked_dummies = pd.get_dummies(df_selected['Embarked'], prefix='Embarked')
df_selected = pd.concat([df_selected, sex_dummies, embarked_dummies], axis=1)
df_selected.drop(['Sex', 'Embarked'], axis=1, inplace=True)

numeric_features = ['Pclass', 'SibSp', 'Parch', 'Fare']
for feature in numeric_features:
    feature_mean = df_selected[feature].mean()
    feature_std = df_selected[feature].std()
    df_selected[feature] = (df_selected[feature] - feature_mean) / feature_std

# División de los datos en conjunto de entrenamiento y prueba
train_size = int(len(df_selected) * 0.8)
train_set = df_selected[:train_size]
test_set = df_selected[train_size:]

X_train = train_set.drop('Fare', axis=1).values
y_train = train_set['Fare'].values
X_test = test_set.drop('Fare', axis=1).values
y_test = test_set['Fare'].values

# Ajuste de la regresión lineal con el algoritmo OLS
X_train = np.c_[np.ones(X_train.shape[0]), X_train]
coefficients = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train

# Evaluación del rendimiento en el conjunto de prueba
X_test = np.c_[np.ones(X_test.shape[0]), X_test]
y_pred = X_test @ coefficients
mse = np.mean((y_test - y_pred) ** 2)

print(f"Error cuadrático medio (MSE): {np.round(mse, 3)}")

Error cuadrático medio (MSE): 1.109


C:\Users\Usuario\AppData\Local\Temp\ipykernel_13436\2991516030.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['CabinAssigned'] = df_selected['Cabin'].notnull().astype(int)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13436\2991516030.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.drop(['Cabin'], axis=1, inplace=True)


## Asignación 4 &#x1F921;

Utilizando las características __Pclass__, __Sex__, __SibSp__, __Parch__, __Fare__, __Cabin__ y __Embarked__, entrene un clasificador lineal para predecir la probabilidad de supervivencia __Survived__ utilizando el algoritmo de gradiente descendente estocástico y la entropía cruzada como función de error. Imprima el arror en cada iteración del gradiente.

In [9]:
import numpy as np

# Selección de características y preprocesamiento de datos
selected_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Survived']
df_selected = df[selected_features]

# Imputar los valores faltantes en la columna 'Fare'
fare_median = df_selected['Fare'].median()
df_selected['Fare'] = df_selected['Fare'].fillna(fare_median)

df_selected['CabinAssigned'] = df_selected['Cabin'].notnull().astype(int)

df_selected.drop(['Cabin'], axis=1, inplace=True)

sex_dummies = pd.get_dummies(df_selected['Sex'], prefix='Sex')
embarked_dummies = pd.get_dummies(df_selected['Embarked'], prefix='Embarked')
df_selected = pd.concat([df_selected, sex_dummies, embarked_dummies], axis=1)
df_selected.drop(['Sex', 'Embarked'], axis=1, inplace=True)

numeric_features = ['Pclass', 'SibSp', 'Parch', 'Fare']
for feature in numeric_features:
    feature_mean = df_selected[feature].mean()
    feature_std = df_selected[feature].std()
    df_selected[feature] = (df_selected[feature] - feature_mean) / feature_std

# División de los datos en conjunto de entrenamiento y prueba
train_size = int(len(df_selected) * 0.8)
train_set = df_selected[:train_size]
test_set = df_selected[train_size:]

X_train = train_set.drop('Survived', axis=1).values
y_train = train_set['Survived'].values
X_test = test_set.drop('Survived', axis=1).values
y_test = test_set['Survived'].values

# Inicialización de los parámetros del modelo
learning_rate = 0.01
num_iterations = 1000
batch_size = 32
num_batches = int(np.ceil(len(X_train) / batch_size))
theta = np.zeros(X_train.shape[1])

# Definición de la función de entropía cruzada
def cross_entropy_loss(y_true, y_pred):
    epsilon = 1e-5
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# Entrenamiento del modelo con gradiente descendente estocástico
for i in range(num_iterations):
    batch_indices = np.random.choice(len(X_train), size=batch_size, replace=False)
    X_batch = X_train[batch_indices]
    y_batch = y_train[batch_indices]

    # Cálculo de la predicción y el error en el lote actual
    y_pred = 1 / (1 + np.exp(-X_batch @ theta))
    loss = cross_entropy_loss(y_batch, y_pred)

    # Actualización de los parámetros del modelo
    gradient = X_batch.T @ (y_pred - y_batch)
    theta -= learning_rate * gradient

    # Impresión del error en cada iteración del gradiente
    print(f"Iteración {i+1}/{num_iterations} - Error: {np.round(loss, 5)}")

# Evaluación del rendimiento en el conjunto
y_pred_test = 1 / (1 + np.exp(-X_test @ theta))
y_pred_test_classes = np.round(y_pred_test)
test_accuracy = np.mean(y_pred_test_classes == y_test)

print(f"Exactitud en el conjunto de prueba: {test_accuracy}")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13436\2634519302.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Fare'] = df_selected['Fare'].fillna(fare_median)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13436\2634519302.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['CabinAssigned'] = df_selected['Cabin'].notnull().astype(int)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13436\2634519302.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

Iteración 1/1000 - Error: 0.69315
Iteración 2/1000 - Error: 0.6697
Iteración 3/1000 - Error: 0.64614
Iteración 4/1000 - Error: 0.63808
Iteración 5/1000 - Error: 0.60852
Iteración 6/1000 - Error: 0.58729
Iteración 7/1000 - Error: 0.56944
Iteración 8/1000 - Error: 0.54947
Iteración 9/1000 - Error: 0.62273
Iteración 10/1000 - Error: 0.57288
Iteración 11/1000 - Error: 0.55948
Iteración 12/1000 - Error: 0.5075
Iteración 13/1000 - Error: 0.52529
Iteración 14/1000 - Error: 0.51338
Iteración 15/1000 - Error: 0.51774
Iteración 16/1000 - Error: 0.53188
Iteración 17/1000 - Error: 0.51764
Iteración 18/1000 - Error: 0.51208
Iteración 19/1000 - Error: 0.49922
Iteración 20/1000 - Error: 0.6041
Iteración 21/1000 - Error: 0.5045
Iteración 22/1000 - Error: 0.46095
Iteración 23/1000 - Error: 0.58434
Iteración 24/1000 - Error: 0.52637
Iteración 25/1000 - Error: 0.55825
Iteración 26/1000 - Error: 0.51035
Iteración 27/1000 - Error: 0.54476
Iteración 28/1000 - Error: 0.55616
Iteración 29/1000 - Error: 0.4182

Iteración 306/1000 - Error: 0.4515
Iteración 307/1000 - Error: 0.55618
Iteración 308/1000 - Error: 0.58941
Iteración 309/1000 - Error: 0.57948
Iteración 310/1000 - Error: 0.52478
Iteración 311/1000 - Error: 0.43523
Iteración 312/1000 - Error: 0.31669
Iteración 313/1000 - Error: 0.55095
Iteración 314/1000 - Error: 0.72831
Iteración 315/1000 - Error: 0.46307
Iteración 316/1000 - Error: 0.44826
Iteración 317/1000 - Error: 0.4438
Iteración 318/1000 - Error: 0.57444
Iteración 319/1000 - Error: 0.68124
Iteración 320/1000 - Error: 0.5238
Iteración 321/1000 - Error: 0.47037
Iteración 322/1000 - Error: 0.4461
Iteración 323/1000 - Error: 0.39081
Iteración 324/1000 - Error: 0.3505
Iteración 325/1000 - Error: 0.45938
Iteración 326/1000 - Error: 0.46517
Iteración 327/1000 - Error: 0.49807
Iteración 328/1000 - Error: 0.50235
Iteración 329/1000 - Error: 0.40605
Iteración 330/1000 - Error: 0.57635
Iteración 331/1000 - Error: 0.56627
Iteración 332/1000 - Error: 0.43526
Iteración 333/1000 - Error: 0.489

Iteración 586/1000 - Error: 0.3416
Iteración 587/1000 - Error: 0.36589
Iteración 588/1000 - Error: 0.42895
Iteración 589/1000 - Error: 0.50311
Iteración 590/1000 - Error: 0.44697
Iteración 591/1000 - Error: 0.42774
Iteración 592/1000 - Error: 0.52934
Iteración 593/1000 - Error: 0.39044
Iteración 594/1000 - Error: 0.41223
Iteración 595/1000 - Error: 0.22929
Iteración 596/1000 - Error: 0.47779
Iteración 597/1000 - Error: 0.51319
Iteración 598/1000 - Error: 0.37688
Iteración 599/1000 - Error: 0.44647
Iteración 600/1000 - Error: 0.496
Iteración 601/1000 - Error: 0.50419
Iteración 602/1000 - Error: 0.36235
Iteración 603/1000 - Error: 0.43573
Iteración 604/1000 - Error: 0.47187
Iteración 605/1000 - Error: 0.34469
Iteración 606/1000 - Error: 0.55153
Iteración 607/1000 - Error: 0.53174
Iteración 608/1000 - Error: 0.58364
Iteración 609/1000 - Error: 0.30842
Iteración 610/1000 - Error: 0.40619
Iteración 611/1000 - Error: 0.4255
Iteración 612/1000 - Error: 0.36902
Iteración 613/1000 - Error: 0.67

Iteración 941/1000 - Error: 0.41531
Iteración 942/1000 - Error: 0.53585
Iteración 943/1000 - Error: 0.40493
Iteración 944/1000 - Error: 0.51184
Iteración 945/1000 - Error: 0.33237
Iteración 946/1000 - Error: 0.59435
Iteración 947/1000 - Error: 0.38553
Iteración 948/1000 - Error: 0.39626
Iteración 949/1000 - Error: 0.45979
Iteración 950/1000 - Error: 0.42114
Iteración 951/1000 - Error: 0.57617
Iteración 952/1000 - Error: 0.52364
Iteración 953/1000 - Error: 0.63382
Iteración 954/1000 - Error: 0.36596
Iteración 955/1000 - Error: 0.53214
Iteración 956/1000 - Error: 0.51681
Iteración 957/1000 - Error: 0.45159
Iteración 958/1000 - Error: 0.36134
Iteración 959/1000 - Error: 0.49743
Iteración 960/1000 - Error: 0.59508
Iteración 961/1000 - Error: 0.44969
Iteración 962/1000 - Error: 0.39857
Iteración 963/1000 - Error: 0.42824
Iteración 964/1000 - Error: 0.59992
Iteración 965/1000 - Error: 0.59243
Iteración 966/1000 - Error: 0.42007
Iteración 967/1000 - Error: 0.35923
Iteración 968/1000 - Error: 